In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import MultiTaskLasso, Lasso
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
import xlsxwriter
import scipy.sparse as sp
from sklearn import linear_model, grid_search, datasets
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from operator import itemgetter

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%cd /home/bbardak/Desktop/Tez/MultipleDisease/sonson/Datasets/

In [ ]:
workbook = xlsxwriter.Workbook('Offset_All_Disease_ElasticNet_Results.xlsx')
worksheet = workbook.add_worksheet()

In [ ]:
all_disease_dataset = pd.read_csv('normalized_cdc_all_disease.csv')
## Disease combination which want to examine
## flu-pertussis, flu-legionellosis, flu-pertussis-meningococal

disease_combinations = [ 
                        ['Flu', 'Pertussis'],
                        ['Flu', 'Legionellosis'],
                        ['Flu', 'Pertussis', 'Meningococcal']
                    ]


gft_dataset = pd.read_csv('normalized_weekly_gft.csv')
gft_columns = ['Idaho', 'Indiana', 'Michigan', 'Iowa', 'Virginia', 'Bati_Virginia', 'New_Mexico', 'Kentucky', 'North_Carolina', 'Alaska', 'Ohio', 'Pennsylvania', 'Texas', 'Tennessee', 'Louisiana', 'Florida', 'Missouri', 'Oklahoma', 'Arkansas', 'Arizona','Nebraska','Wyoming','Delaware','Illinois','Colorado','New_York','Maryland','Alabama','Kansas','Utah']

wiki_dataset = pd.read_csv('normalized_daily_wiki.csv')
wiki_columns = ['en-Common_cold','en-Human_flu','en-Influenza','en-Influenza-like_illness','en-Influenza_A_virus','en-Influenza_A_virus_subtype_H1N1','en-Influenzavirus_A','en-Influenzavirus_C','en-Oseltamivir','en-Zanamivir']



In [ ]:
WIKI_STARTING_DATE = []
WIKI_ENDING_DATE = []

for index,i in enumerate(wiki_dataset['Unnamed: 0']):
    if i == '2011-01-01':
        WIKI_STARTING_DATE.append(i)
        WIKI_STARTING_DATE.append(index)
    if i == '2014-01-10':
        WIKI_ENDING_DATE.append(i)
        WIKI_ENDING_DATE.append(index)
#a = wiki.groupby(np.arange(len(wiki))//7).sum()


## OFFSET FOR GFT DATA
GFT_STARTING_DATE = []
GFT_ENDING_DATE = []


for index,i in enumerate(gft_dataset['Unnamed: 0']):
    if i == '2011-01-02':
        GFT_STARTING_DATE.append(i)
        GFT_STARTING_DATE.append(index)
    if i == '2014-01-05':
        GFT_ENDING_DATE.append(i)
        GFT_ENDING_DATE.append(index)
        
wiki_offset = [-21,-19,-17,-15,-13,-11,-9,-7,-5,-3,-1,0]
gft_offset = np.arange(-2,1,1)
disease_offset = np.arange(-2,1,1)

gft_start_range = GFT_STARTING_DATE[1]
gft_end_range = GFT_ENDING_DATE[1]+1

wiki_start_range = WIKI_STARTING_DATE[1]
wiki_end_range = WIKI_ENDING_DATE[1]+1


In [ ]:
for gft_offset_index in gft_offset:
    offset_weekly_gft = gft_dataset[GFT_STARTING_DATE[1]+gft_offset_index:GFT_ENDING_DATE[1]+gft_offset_index]
    gft_selected_columns_data = offset_weekly_gft.as_matrix(columns=gft_columns)

In [ ]:
disease = ['Flu', 'Pertussis', 'Meningococcal']

col = 0
row = 1

sayac = 0

for wiki_offset_index in wiki_offset:
    if sayac == 3: break
    offset_weekly_wiki = wiki_dataset[wiki_start_range+wiki_offset_index:wiki_end_range+wiki_offset_index].groupby(np.arange(len(wiki_dataset[wiki_start_range+wiki_offset_index:wiki_end_range+wiki_offset_index]))//7).sum()    wiki_selected_columns_data = offset_weekly_wiki.as_matrix(columns=wiki_columns)
    
    for gft_offset_index in gft_offset:
        if sayac == 3: 
            break
        offset_weekly_gft = gft_dataset[gft_start_range+gft_offset_index:gft_end_range+gft_offset_index]
        gft_selected_columns_data = offset_weekly_gft.as_matrix(columns=gft_columns)
   
        c1 = sp.csr_matrix(gft_selected_columns_data)
        c2 = sp.csr_matrix(wiki_selected_columns_data)
        h = sp.hstack((c1, c2), format='csr')
        train_data = h.A

        for first_disease_offset in disease_offset:
            if sayac == 3:
                break
            for second_disease_offset in disease_offset:
                sayac = sayac + 1
                if sayac == 3: 
                    break
                first_disease_data = all_disease_dataset['Pertussis'][first_disease_offset + 2: len(all_disease_dataset['Pertussis']) + first_disease_offset]
                second_disease_data = all_disease_dataset['Meningococcal'][second_disease_offset + 2: len(all_disease_dataset['Meningococcal']) + second_disease_offset]
                flu_data = all_disease_dataset['Flu'][2:]
                
                first_disease_data = first_disease_data.reset_index()
                second_disease_data = second_disease_data.reset_index()
                flu_data = flu_data.reset_index()
                
                editted_disease_frame = pd.concat([first_disease_data, second_disease_data, flu_data], axis=1)
                del editted_disease_frame['index']
                
                ground_truth = editted_disease_frame.as_matrix(columns=disease)
                
                offset_day =  str(wiki_offset_index)+","+str(gft_offset_index)+","+str(first_disease_offset)+","+str(second_disease_offset)
                print offset_day
                r2, mse, mae, rep = getScore(train_data, ground_truth, 3) ## 3 equals to number of disease
                
                col = 0
                worksheet.write(row, col, str(offset_day))
                col += 1

                ## write r2 avarage scores for each disase
                for i in range(len(disease)): 
                    result = np.asarray(r2[i::len(disease)]).sum() / len(r2[i::len(disease)])
                    worksheet.write(row,col,str(result))
                    col +=1
                for i in range(len(disease)): 
                    result = r2[i::len(disease)]
                    worksheet.write(row,col,str(result)) 
                    col +=1
                ## write mse avarage scores for each disease, then write each mse scores
                for i in range(len(disease)): 
                    result = np.asarray(mse[i::len(disease)]).sum() / len(mse[i::len(disease)])
                    worksheet.write(row,col,str(result))
                    col +=1
                for i in range(len(disease)): 
                    result = mse[i::len(disease)] 
                    worksheet.write(row, col,str(result) )
                    col += 1
                                       
                ## write mae avarage scores for each disease, then write each mae scores
                for i in range(len(disease)): 
                    result = np.asarray(mae[i::len(disease)]).sum() / len(mae[i::len(disease)])
                    worksheet.write(row, col, str(result))
                    col += 1
                
                for i in range(len(disease)): 
                    result = mae[i::len(disease)] 
                    worksheet.write(row, col, str(result))
                    col += 1
                ## write mean validation, std, best parameters
                worksheet.write(row, col, str(rep))
                row += 1

workbook.close()                                

In [ ]:
# Try on 1st disease combination: Flu Pertussis Meningococcal

In [ ]:
for each_disease_combination in disease_combinations:
    print each_disease_combination

In [ ]:
for i in disease_combinations:
    print i
    print all_disease_dataset.as_matrix(i)

In [ ]:
def getScore(X,y, numberOfDisease):
    data = X
    gt = y
    r2 = []
    mse = []
    mae = []
    rep = []
    parameters = {"alpha": [1, 0.1, 0.01, 0.001, 0.0001],
                  "l1_ratio": [0, 0.25, 0.5,0.75,1],
              "fit_intercept": [True],
              "max_iter":[1000],
              "normalize":[True, False],
              "selection":['cyclic', 'random'],
              "tol":[0.0001, 0.001]}
    
    #cv = cross_validation.KFold(len(train), n_folds=10, shuffle=True)
    #for data,gt in cv:
       #print data, gt
    s = 0
    for random_generator in range(1,11):
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(data, gt, test_size=0.3, random_state=random_generator)
        mtl = linear_model.MultiTaskElasticNet()
        grid_search = GridSearchCV(mtl, param_grid=parameters, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        predictions = grid_search.predict(X_test)
        for i in range(0,numberOfDisease):
            s += 1
            r2.append(r2_score(y_test[:,i], predictions[:,i]))
            mse.append(mean_squared_error(y_test[:,i], predictions[:,i]))
            mae.append(mean_absolute_error(y_test[:,i], predictions[:,i]))

        temp = report(grid_search.grid_scores_)
        rep.append(temp)
    return r2, mse, mae, rep

In [ ]:
def report(grid_scores, n_top=1):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    l = []
    for i, score in enumerate(top_scores):
        
        #print("Model with rank: {0}".format(i + 1))
        #print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
        #     score.mean_validation_score,
        #      np.std(score.cv_validation_scores)))
        #print("Parameters: {0}".format(score.parameters))
        #print("")
        l.append(score.mean_validation_score)
        l.append(np.std(score.cv_validation_scores))
        l.append(score.parameters)
    return l